In [1]:
from bs4 import BeautifulSoup
from selenium  import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.touch_actions import TouchActions
import time
import pandas as pd
driver = webdriver.Chrome(ChromeDriverManager().install())




Checking for linux64 chromedriver:76.0.3809.126 in cache
Driver found in /home/shakti/.wdm/chromedriver/76.0.3809.126/linux64/chromedriver


In [2]:
driver.get('https://www.prokabaddi.com')
time.sleep(1)
standings=driver.find_element_by_link_text("Standings")
standings.click()
#### TO WRITE DRIVER PAGE AS HTML
#soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("StandingsPage1.html", "w") as file:
#     file.write(str(soup))
time.sleep(2)
tableCOntent=driver.find_element_by_class_name("sipk-standing-page ")
table=tableCOntent.text
table=table.split('\n')[1:]
numberOfColumns=8
numberOfTeams=12
columns=table[:numberOfColumns]
tablevalues=[]
for i in range(numberOfColumns,len(table),numberOfColumns):
    tablevalues.append(table[i:i+numberOfColumns])
tablevalues=tablevalues[:numberOfTeams]
LeagueTable=pd.DataFrame.from_records(tablevalues,columns=columns)
LeagueTable.to_csv("./Leaguetable.csv")

IMPORTANT GUIDE FOR SELENIUM:
https://www.pluralsight.com/guides/web-scraping-with-selenium

In [4]:
driver.get('https://www.flashscore.com.au/kabaddi/india/pro-kabaddi/results/')
time.sleep(1)
fixtureHistory=driver.find_element_by_id("live-table")
fixtureHistory=fixtureHistory.text.split('\n')[3:]
# #### TO WRITE DRIVER PAGE AS HTML
# soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("ResultsPage.html", "w") as file:
#     file.write(str(soup))

columns=['Date','HomeTeam','AwayTeam','Total_Points_HomeTeam','Total_Points_AwayTeam','FirstHalf_Points_HomeTeam','FirstHalf_Points_AwayTeam','SecondHalf_Points_HomeTeam','SecondHalf_Points_AwayTeam']
tablevalues=[]
for i in range(0,len(fixtureHistory),len(columns)):
    tablevalues.append(fixtureHistory[i:i+len(columns)])
ResultsTable=pd.DataFrame.from_records(tablevalues,columns=columns)
ResultsTable.to_csv('./PointsTable.csv')

In [5]:
driver.get('https://www.flashscore.com.au/kabaddi/india/pro-kabaddi/fixtures/')
time.sleep(1)
fixtureUpcoming=driver.find_element_by_id("live-table").text.split('\n')[3:]
fixtureUpcoming
columns=['date','HomeTeam','AwayTeam','Drop1','Drop2']
tablevalues=[]
for i in range(0,len(fixtureUpcoming),len(columns)):
    tablevalues.append(fixtureUpcoming[i:i+len(columns)])
UpcomingTable=pd.DataFrame.from_records(tablevalues,columns=columns)
UpcomingTable=UpcomingTable.drop(['Drop1','Drop2'],axis=1)
UpcomingTable.to_csv('./Upcomingfixtures.csv')

# #### TO WRITE DRIVER PAGE AS HTML
# soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("FixturesPage.html", "w") as file:
#     file.write(str(soup))

In [6]:
driver.get('https://www.mykhel.com/kabaddi/pro-kabaddi-team-players/')
#time.sleep(1) --> ENABLE IF SLOW NET
list_of_team_hrefs=[]
TOTALPLAYERDATA=[]
content_blocks = driver.find_elements_by_class_name("pk-squads")
for block in content_blocks:
    elements = block.find_elements_by_tag_name("a")
    for el in elements:
        tmpel=el.get_attribute("href")
        if('squad' in tmpel ):
            list_of_team_hrefs.append(tmpel)
for teamlink in list_of_team_hrefs:
    print(teamlink)
    driver.get(teamlink)
    #time.sleep(1) --> ENABLE IF SLOW NET
    list_of_hrefs = []
    content_blocks = driver.find_elements_by_class_name("pk-squads__block")
    for block in content_blocks:
        elements = block.find_elements_by_tag_name("a")
        for el in elements:
            list_of_hrefs.append(el.get_attribute("href"))
    for k in range(len(list_of_hrefs)):
        driver.get(list_of_hrefs[k])
        #time.sleep(1) --> ENABLE IF SLOW NET
        stats= driver.find_elements_by_class_name("pk-team__playerstats")
        name = driver.find_elements_by_class_name("pk-heading")[0].text.split(' ')
        playstats=stats[0].text.split('\n')
        playerbio=playstats[:6]
        playerskills=playstats[7:]
        columns=[]
        data=[]
        columns.append('Name')
        if(name[1]!='Pro'):
            data.append(name[0]+' '+name[1])
        else:
            data.append(name[0])
        print(data[0])
        for i in playerbio:
            j=i.split(':')
            columns.append(str(j[0]))
            data.append(str(j[1]).strip(' '))
        playerskills.remove('RAID')
        playerskills.remove('TACKLES')
        for i in range(len(playerskills)):
            if(playerskills[i]=='DISCIPLINARY'):
                break
            if(i%2 ==0):
                    data.append(float(playerskills[i]))
            else:
                columns.append(str(playerskills[i]))  
        TOTALPLAYERDATA.append(data)
PlayerData=pd.DataFrame(TOTALPLAYERDATA,columns=columns)
PlayerData.to_csv('./PlayerData.csv')
    

https://www.mykhel.com/kabaddi/pro-kabaddi-bengal-warriors-squad-t4/
Maninder Singh
K. Prapanjan
Mohammad Taghi
Rakesh Narwal
Ravindra Ramesh
Sukesh Hegde
Naveen Narwal
Amit
Baldev Singh
Jeeva Kumar
Viraj Vishnu
Adarsh T
Dharmendra Singh
Rinku Narwal
Amir Santosh
Avinash A.
Mohammad Esmaeil
Mayur Shivtarkar
Sourabh Tanaji
Sunil Manik
https://www.mykhel.com/kabaddi/pro-kabaddi-bengaluru-bulls-squad-t1/
Rohit Kumar
Banty
Lal Mohar
Pawan Kumar
Sumit Singh
Vinod Kumar
Mohit Sehrawat
Raju Lal
Vijay Kumar
Mahender Singh
Aman
Sandeep
Saurabh Nandal
Ajay
Amit Sheoran
Ankit
Ashish Kumar
Sanjay Shrestha
https://www.mykhel.com/kabaddi/pro-kabaddi-dabang-delhi-k-c-squad-t2/
Joginder Singh
Aman Kadian
Chandran Ranjit
Naveen Kumar
Neeraj Narwal
Sumit Kumar
Mohit
Vishal Mane
Pratik Patil
Ravinder Pahal
Anil Kumar
Saeid Ghaffari
Satywan
Sumit
Sombir
Balram
Meraj Sheykh
Vijay
https://www.mykhel.com/kabaddi/pro-kabaddi-gujarat-fortunegiants-squad-t31/
Sunil Kumar
Abolfazl Maghsodloumahali
Gurvinder Sing